In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [20]:
cols = pickle.load(open('../cjy/input/test_matrix/m_622_cols.p','rb'))
cols[:5]
coo = pickle.load(open('../cjy/input/test_matrix/m_622.p','rb'))
coo = coo.tocoo(copy=False)
m_622_frame = pd.DataFrame({'index': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['index', 'col', 'data']].sort_values(['index', 'col']
                 ).reset_index(drop=True)
m_622_frame.head()

['full_order', 'b_3758640', 'b_3155503', 'b_56331008', 'b_33489263']

,index,col,data
0,0,0,1138172.0
1,0,948,-82.0
2,0,1483,-85.0
3,0,2105,-61.0
4,0,3453,-77.0


# Similarity

In [3]:
import math

In [4]:
def Cosine(x, y):
    x = np.array(x)
    y = np.array(y)
    return float(x.dot(y)/(math.sqrt((x**2).sum()) * math.sqrt((y**2).sum())))

In [5]:
def Multiply(x, y):
    sum_xy = 0.0
    for i in range(len(x)):
        temp = x[i]*y[i]
        sum_xy = sum_xy + temp
    return float(sum_xy)

def Pearson(x, y):
    n = len(x)
    sum_x = sum(x)
    sum_y = sum(y)
    sum_xy = Multiply(x, y)
    sum_x2 = sum([i**2 for i in x])
    sum_y2 = sum([j**2 for j in y])
    m = sum_xy - (float(sum_x) * float(sum_y)/n)
    d = math.sqrt((sum_x2-float(sum_x**2)/n) * (sum_y2-float(sum_y**2)/n))
    return float(m/d)

In [30]:
def shop_sim(mall_frame, x, y):
    temp1 = mall_frame[mall_frame['index'] == x].index
    temp2 = mall_frame[mall_frame['index'] == y].index
    dot = 0
    xx = 0
    for i in temp1:
        yy = 0
        if mall_frame.iloc[i].col != 0:
            x_stre = mall_frame.iloc[i]['data']
            xx = xx + x_stre**2
#             print('xx :' + str(xx))
            for j in temp2:
                if mall_frame.iloc[j].col == mall_frame.iloc[i].col:
                    y_stre = mall_frame.iloc[j]['data']
                    yy = yy + y_stre**2
#                     print('yy :' + str(yy))
                    dot = dot + x_stre * y_stre
    if dot == 0:
        return 0
    else:
        return float((1000 * dot)/(xx * yy))

In [7]:
shop_sim(m_622_frame, 2, 3)

0.0002926100565698812

# Shops Sim

In [8]:
user = pd.read_csv('../input/训练数据-ccf_first_round_user_shop_behavior.csv')
shop = pd.read_csv('../input/训练数据-ccf_first_round_shop_info.csv')
test = pd.read_csv('../input/AB榜测试集-evaluation_public.csv')

user.shape
user.head()
shop.shape
shop.head()
test.shape
test.head()

(1138015, 6)

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


(8477, 6)

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
1,s_133,c_6,121.134362,31.197511,58,m_6587
2,s_251,c_38,121.000505,30.907667,34,m_5892
3,s_372,c_30,119.864982,26.659876,44,m_625
4,s_456,c_26,122.594243,31.581499,44,m_3839


(483931, 7)

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [9]:
user = user.merge(shop, left_on='shop_id', right_on='shop_id', how='left')
user['row_id'] = -1

del shop

user = user[['row_id', 'user_id', 'mall_id', 'wifi_infos', 'shop_id']]
test = test[['row_id', 'user_id', 'mall_id', 'wifi_infos']]

user.shape
user.head()
test.shape
test.head()

(1138015, 5)

,row_id,user_id,mall_id,wifi_infos,shop_id
0,-1,u_376,m_1409,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718
1,-1,u_376,m_1409,b_6396480|-67|false;b_56328155|-73|false;b_411...,s_2871718
2,-1,u_1041,m_4079,b_8006367|-78|false;b_2485110|-52|false;b_3005...,s_181637
3,-1,u_1158,m_6587,b_26250579|-73|false;b_26250580|-64|false;b_26...,s_609470
4,-1,u_1654,m_3005,b_39004150|-66|false;b_39004148|-58|false;b_21...,s_3816766


(483931, 4)

,row_id,user_id,mall_id,wifi_infos
0,118742,u_30097142,m_3916,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [10]:
test['shop_id'] = -1

full_data = pd.concat([user, test])

full_data.shape
full_data.head()

(1621946, 5)

,row_id,user_id,mall_id,wifi_infos,shop_id
0,-1,u_376,m_1409,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718
1,-1,u_376,m_1409,b_6396480|-67|false;b_56328155|-73|false;b_411...,s_2871718
2,-1,u_1041,m_4079,b_8006367|-78|false;b_2485110|-52|false;b_3005...,s_181637
3,-1,u_1158,m_6587,b_26250579|-73|false;b_26250580|-64|false;b_26...,s_609470
4,-1,u_1654,m_3005,b_39004150|-66|false;b_39004148|-58|false;b_21...,s_3816766


In [11]:
del user
del test

In [12]:
full_data.wifi_infos = full_data.wifi_infos.apply(lambda x : x.split(';'))
full_data['wifi_num'] = full_data.wifi_infos.apply(len)

full_data.head()

,row_id,user_id,mall_id,wifi_infos,shop_id,wifi_num
0,-1,u_376,m_1409,"[b_6396480|-67|false, b_41124514|-86|false, b_...",s_2871718,10
1,-1,u_376,m_1409,"[b_6396480|-67|false, b_56328155|-73|false, b_...",s_2871718,10
2,-1,u_1041,m_4079,"[b_8006367|-78|false, b_2485110|-52|false, b_3...",s_181637,10
3,-1,u_1158,m_6587,"[b_26250579|-73|false, b_26250580|-64|false, b...",s_609470,10
4,-1,u_1654,m_3005,"[b_39004150|-66|false, b_39004148|-58|false, b...",s_3816766,10


In [13]:
full_data.reset_index(drop=True, inplace=True)

full_data.tail()

,row_id,user_id,mall_id,wifi_infos,shop_id,wifi_num
1621941,483927,u_63919729,m_7523,"[b_30877374|-71|false, b_8210864|-61|false, b_...",-1,10
1621942,483928,u_63920157,m_4828,"[b_8685813|-79|false, b_8685969|-74|false, b_1...",-1,10
1621943,483929,u_63920157,m_4828,"[b_49160464|-65|false, b_34498163|-82|false, b...",-1,8
1621944,483930,u_63920157,m_4828,"[b_45089046|-47|true, b_49160464|-65|false, b_...",-1,3
1621945,483931,u_63920806,m_7973,"[b_25163301|-69|false, b_39872252|-57|false, b...",-1,10


In [14]:
def wifi_list(x):
    a = []
    while len(x) != 0:
        b = x.pop(0)
        a.append(b.split('|'))
    return a

In [15]:
full_data.wifi_infos = full_data.wifi_infos.apply(wifi_list)

full_data.head()

,row_id,user_id,mall_id,wifi_infos,shop_id,wifi_num
0,-1,u_376,m_1409,"[[b_6396480, -67, false], [b_41124514, -86, fa...",s_2871718,10
1,-1,u_376,m_1409,"[[b_6396480, -67, false], [b_56328155, -73, fa...",s_2871718,10
2,-1,u_1041,m_4079,"[[b_8006367, -78, false], [b_2485110, -52, fal...",s_181637,10
3,-1,u_1158,m_6587,"[[b_26250579, -73, false], [b_26250580, -64, f...",s_609470,10
4,-1,u_1654,m_3005,"[[b_39004150, -66, false], [b_39004148, -58, f...",s_3816766,10


In [16]:
train = full_data[full_data.row_id == -1]
test = full_data[full_data.row_id != -1]

del full_data

In [18]:
train_mall_ids = set(train.mall_id)
test_mall_ids = set(test.mall_id)

In [17]:
for mall_id in train_mall_ids:
    cols = pickle.load(open('../cjy/input/train_matrix/' + str(mall_id) + '_cols.p','rb'))
    coo = pickle.load(open('../cjy/input/train_matrix/' + str(mall_id) + '.p','rb'))
    coo = coo.tocoo(copy=False)
    mall_frame = pd.DataFrame({'index': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['index', 'col', 'data']].sort_values(['index', 'col']
                 ).reset_index(drop=True)
    index = set(mall_frame['index'])
    with pd.ExcelWriter('./output/train_shop_sim.xlsx') as writer:
        temp = pd.DataFrame()
        for i in index:
            for j in index:
                s = pd.Series({'sim' : shop_sim(mall_frame, i, j), 'i' : i, 'j' : j})
                temp.append(s, ignore_index=True)
        temp.to_excel(writer, sheet_name=str(i) + ' ' + str(j))

NameError: name 'train_mall_ids' is not defined

In [2]:
import pickle

In [ ]:
for mall_id in train_mall_ids:
    cols = pickle.load(open('../cjy/input/train_matrix/' + str(mall_id) + '_cols.p','rb'))
    coo = pickle.load(open('../cjy/input/train_matrix/' + str(mall_id) + '.p','rb'))
    coo = coo.tocoo(copy=False)
    mall_frame = pd.DataFrame({'index': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['index', 'col', 'data']].sort_values(['index', 'col']
                 ).reset_index(drop=True)
    index = set(mall_frame['index'])
    for i in index:
        for j in index:
            sim = shop_sim(mall_frame, i, j)
            if sim != 0:
                s = {'i' : i, 'j' : j, 'sim' : sim}
                pickle.dump(s, open('./output/train/' + str(mall_id) + '.p', 'wb'))

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars


In [29]:
m = pickle.load(open('./output/train/m_7168.p','rb'))
m

{'i': 0, 'j': 2340, 'sim': 3.667236728180836e-05}

# k Sim Shop

In [32]:
def mall_wifi_dateframe(mall_id, path):
    cols = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'_cols.p','rb'))
    coo = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'.p','rb'))
    coo = coo.tocoo(copy=False)
    coo_group = pd.DataFrame({'row': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['row', 'col', 'data']].sort_values(['row', 'col']
                 ).reset_index(drop=True).groupby('col')
    df = pd.DataFrame(-113 * np.ones((max(coo.row)+1, len(cols))), columns=cols, dtype=np.int64)
    tmp = coo_group.apply(lambda x:x.row.values)
    print(tmp)
    for col,rows,values in zip(tmp.index.tolist(),tmp.values,
                                coo_group.apply(lambda x:x.data.tolist())):
        df.iloc[rows,col]=values
    return df

In [33]:
mall = mall_wifi_dateframe('m_622', 'train_matrix')
mall.head()

col
0       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
2                                            [1411, 8926]
3                                              [555, 559]
4                                                  [2935]
5                                                  [2493]
6                                                  [9608]
7                                                  [7737]
8                                      [1674, 1675, 1676]
9                                                  [9085]
10                                                 [5502]
11                                           [1319, 7633]
12      [2485, 4218, 4223, 4227, 4228, 4229, 4234, 423...
13                                                 [5690]
14                         [2227, 4261, 5784, 5785, 9003]
15                               [9356, 9357, 9359, 9363]
16                                     [1359, 1703, 7107]
19                                                [10512]
20        

,full_order,b_3758640,b_3155503,b_56331008,b_33489263,b_18655345,b_9091070,b_6724355,b_49662085,b_19188542,...,b_18304278,b_41824143,b_55555065,b_23622024,b_19054820,b_20952577,b_39241838,b_40230368,b_53654217,b_56549844
0,26,-113,-113,-113,-113,-113,-113,-113,-113,-113,...,-113,-113,-113,-113,-113,-113,-113,-113,-113,-113
1,209,-113,-113,-113,-113,-113,-113,-113,-113,-113,...,-113,-113,-113,-113,-113,-113,-113,-113,-113,-113
2,299,-113,-113,-113,-113,-113,-113,-113,-113,-113,...,-113,-113,-113,-113,-113,-113,-113,-113,-113,-113
3,403,-113,-113,-113,-113,-113,-113,-113,-113,-113,...,-113,-113,-113,-113,-113,-113,-113,-113,-113,-113
4,1152,-113,-113,-113,-113,-113,-113,-113,-113,-113,...,-113,-113,-113,-113,-113,-113,-113,-113,-113,-113


In [16]:
# 列名
cols = pickle.load(open('../cjy/input/train_matrix/m_5767_cols.p','rb'))
cols[:5]
# 读数据
coo=pickle.load(open('../cjy/input/train_matrix/m_5767.p','rb'))
coo = coo.tocoo(copy=False)
pd.DataFrame({'index': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['index', 'col', 'data']].sort_values(['index', 'col']
                 ).reset_index(drop=True)
del cols
del coo

['full_order', 'b_54288784', 'b_30237522', 'b_29294377', 'b_9656937']

,index,col,data
0,0,0,622.0
1,0,122,-79.0
2,0,794,-79.0
3,0,1151,-79.0
4,0,1315,-56.0
5,0,2365,-70.0
6,0,2555,-75.0
7,0,2998,-53.0
8,0,3042,-53.0
9,0,3155,-83.0


In [2]:
k = 4

In [3]:
m = pickle.load(open('./output/train/m_5767.p','rb'))
m
del m

FileNotFoundError: [Errno 2] No such file or directory: './output/train/m_5767.p'

In [4]:
m = pickle.load(open('./output/train_user/m_6587.p','rb'))
m
del m

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,3,147,148,149,150,151,230,562,563,587,...,1136617,1136858,1137003,1137062,1137063,1137687,1137726,1137727,1138013,1138014
1,26799,190169,642001,835990,296683,911335,1295,13211,692634,525836,...,940083,1015116,76786,858700,824497,39504,14025,14025,182402,39504
2,302024,190172,169784,1048496,73651,321101,361691,618581,132012,752531,...,1027783,1091251,76787,335439,213669,386487,799125,100067,688373,596377
3,820486,452918,1053513,683004,239908,243313,763022,1060087,876315,116116,...,159319,340844,717431,1058138,317754,543197,551956,799125,533629,823555
4,295622,640718,482587,683006,667549,549291,101121,239186,539558,889703,...,749072,584110,717433,585755,687191,706785,698926,551956,359717,549484
5,780872,26799,804830,408018,1050477,195245,321098,1016795,554634,154726,...,490137,738402,927754,539559,539560,836514,307400,1041954,113118,140176
6,691385,747657,1112797,682444,85201,155602,933176,207386,843228,904293,...,1096105,1124409,1007277,621002,515764,574640,412245,698926,39504,776201
7,584109,1022912,204777,1051906,457866,305398,260468,157346,109788,1060159,...,711996,1124411,651707,876315,1124914,552237,277043,307400,705369,314416
8,760899,47631,633195,1051904,726723,274009,692370,621002,823833,793213,...,850971,680138,397265,759775,665272,1130250,461816,412245,893184,205346
9,328455,464521,324956,591554,320588,360546,540705,1058139,77507,1131495,...,858463,1027438,718576,301393,101905,1045609,603182,239715,142757,99579


In [45]:
m.iloc[0].sort_values(ascending=False).index[1]

2727

# Sparse Matrix

In [21]:
from scipy.sparse import csr_matrix, csc_matrix, dok_matrix

In [19]:
m_csc = csc_matrix(m)
spa = m_csc.nonzero()
raw = spa[0]
col = spa[1]

In [17]:
raw
col
m[0, 0]

array([    0,     0,     0, ..., 11798, 11798, 11798], dtype=int32)

array([    0,    20,    23, ..., 11753, 11775, 11798], dtype=int32)

1.0

In [22]:
m_dok = dok_matrix(m)
m_dok

<11799x11799 sparse matrix of type '<class 'numpy.float64'>'
	with 20837041 stored elements in Dictionary Of Keys format>

In [23]:
pd.DataFrame(m_dok, index=range(len(m_dok)))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-57793de7773c>", line 1, in <module>
    pd.DataFrame(m_dok, index=range(len(m_dok)))
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/pandas/core/frame.py", line 275, in __init__
    mgr = self._init_dict(data, index, columns, dtype=dtype)
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/pandas/core/frame.py", line 409, in _init_dict
    arrays = [data[k] for k in keys]
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/pandas/core/frame.py", line 409, in <listcomp>
    arrays = [data[k] for k in keys]
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/scipy/sparse/dok.py", line 151, in __getitem__
    j_intlike = isintlike(j)
  File "/usr/local/miniconda2/envs/py35/lib/python3.

KeyboardInterrupt: 

In [31]:
data = pd.DataFrame({'raw' : raw, 'col' : col})

def get_value(x):
    return m_csc[x['col'], x['raw']]

data['value'] = data.apply(get_value, axis=1)

KeyboardInterrupt: 

In [32]:
data.head()

,col,raw
0,0,0
1,20,0
2,23,0
3,46,0
4,58,0


# semi-SMO

In [2]:
from scipy.sparse import csr_matrix

In [ ]:
matrix_dict = {}
def create_wifi_matrix(group):
    mall_id = group.mall_id.values[0]
    if mall_id not in matrix_dict:
        cols = ['full_order']
        cols+=(list(mall_wifi_cnt[mall_id].keys()))
        df = pd.DataFrame(np.zeros((group.shape[0], len(mall_wifi_cnt[mall_id])+1)), columns=cols)
        print(mall_id,df.shape)
        for i in range(group.shape[0]):
            ser = group.iloc[i]
            df.loc[i,'full_order'] = ser.full_order
            df.loc[i,[item[0] for item in ser.wifi_infos]] = [int(item[1]) for item in ser.wifi_infos]
        
        value = scipy.sparse.csr_matrix(df.values)
        cols = df.columns.tolist()

        pickle.dump(value, open('./input/semi_SMO/'+mall_id+'.p','wb'))
        pickle.dump(cols, open('./input/semi_SMO/'+mall_id+'_cols.p','wb'))
        
        matrix_dict[mall_id] = 1
        return df.shape
    else:
        return (-1,-1)
    
df_full.iloc[train_user.shape[0]:].groupby('mall_id').apply(create_wifi_matrix)